# How to generate error bars for 2Q-GST


In [1]:
import pygsti
import time

#If we were using MPI
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
comm = None

#Load the 2-qubit results (if you don't have this directory, run the 2Q-GST example)
results = pygsti.io.read_results_from_dir("example_files/My2QExample", "StandardGST")

In [2]:
# error bars in reports require the presence of a fully-initialized
# "confidence region factory" within the relevant Estimate object.
# In most cases "fully-initialized" means that a Hessian has been 
# computed and projected onto the non-gauge space.
start = time.time()

# Choose forward simulator and how computation should be split up ("distribution parameters"). Here are some examples:
#results.estimates['CPTP'].models['stdgaugeopt'].sim = "map"  # reduces memory requirements
results.estimates['CPTP'].models['stdgaugeopt'].sim = pygsti.forwardsims.MapForwardSimulator(num_atoms=20, param_blk_sizes=(10,10))
#results.estimates['CPTP'].models['stdgaugeopt'].sim = pygsti.forwardsims.MatrixForwardSimulator(num_atoms=20, param_blk_sizes=(10,10))

# initialize a factory for the 'go0' gauge optimization within the 'default' estimate
crfact = results.estimates['CPTP'].add_confidence_region_factory('stdgaugeopt', 'final')
crfact.compute_hessian(comm=comm, mem_limit=400*(1024.0)**3) #optionally use multiple processors & set memlimit
crfact.project_hessian('intrinsic error')

end = time.time()
print("Total time=%f hours" % ((end - start) / 3600.0))

Layout creation w/mem limit = 400.00GB
MapLayout: 1 processors divided into 1 x 1 x 1 (= 1) grid along circuit and parameter directions.
   20 atoms, parameter block size limits (10, 10)
*** Distributing 20 atoms to 1 atom-processing groups (1 cores) ***
    More atom-processors than hosts: each host gets ~1 atom-processors
    Atom-processors already occupy a single node, dividing atom-processor into 1 param-processors.
*** Divided 1-host atom-processor (~1 procs) into 1 param-processing groups ***
    Param-processors already occupy a single node, dividing param-processor into 1 param2-processors.
*** Divided 1-host param-processor (~1 procs) into 1 param2-processing groups ***
rank 0: 9.05991e-06s: beginning atom 1/20, atom-size (#circuits) = 196
rank 0: 0.887687s: block 1/36864, atom 0/20, atom-size (#circuits) = 196
rank 0: 1.80597s: block 2/36864, atom 0/20, atom-size (#circuits) = 196
rank 0: 2.7275s: block 3/36864, atom 0/20, atom-size (#circuits) = 196
rank 0: 3.6738s: block 4

KeyboardInterrupt: 

Note above cell was executed for demonstration purposes, and was **keyboard-interrupted intentionally** since it would have taken forever on a single processor.

In [ ]:
#write results back to disk
results.write()